# Day 2: Data Preprocessing for Tabular Model (Rossmann)

## 1. Objective
- Load and inspect Rossmann dataset.
- Merge datasets and handle missing values.
- Identify categorical and continuous features.

## 2. Key Steps
- Load `train.csv` and `store.csv`
- Merge them on `Store`
- Inspect nulls, data types, and structure

In [19]:
# Import required packages
from fastai.tabular.all import *
import pandas as pd
import numpy as np
from pathlib import Path

# Set data path
path = Path('../data/rossmann')
assert path.exists(), "Dataset path not found."

# Load CSVs
train_df = pd.read_csv(path/'train.csv', low_memory=False)
store_df = pd.read_csv(path/'store.csv')

print("Files loaded")
train_df.shape, store_df.shape

Files loaded


((1017209, 9), (1115, 10))

In [20]:
# Merge train with store metadata on 'Store'
df = pd.merge(train_df, store_df, how='left', on='Store')

# Peek at combined data
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [21]:
df.shape

(1017209, 18)

## 3. Results
- Combined dataset has 1017209 rows and 18 columns.
- Next: clean date column, identify missing values, and define variable types.

In [22]:
print(f"{'Column':<20} {'Type':<15} {'Example'}")
print("-" * 60)
for col in df.columns:
    dtype = df[col].dtype
    example = df[col].dropna().iloc[0] if df[col].notna().any() else "NaN"
    print(f"{col:<20} {str(dtype):<15} {str(example)}")

Column               Type            Example
------------------------------------------------------------
Store                int64           1
DayOfWeek            int64           5
Date                 object          2015-07-31
Sales                int64           5263
Customers            int64           555
Open                 int64           1
Promo                int64           1
StateHoliday         object          0
SchoolHoliday        int64           1
StoreType            object          c
Assortment           object          a
CompetitionDistance  float64         1270.0
CompetitionOpenSinceMonth float64         9.0
CompetitionOpenSinceYear float64         2008.0
Promo2               int64           0
Promo2SinceWeek      float64         13.0
Promo2SinceYear      float64         2010.0
PromoInterval        object          Jan,Apr,Jul,Oct


## 4. Preprocessing
We now convert the `Date` column to datetime, extract date features, and handle missing values.
We'll also define which columns are categorical vs. continuous for use in fastai.

In [23]:
# Convert Date column
df['Date'] = pd.to_datetime(df['Date'])

# Extract date features (fastai-style)
add_datepart(df, 'Date', drop=True)

# Look at missing values
df.isna().sum()[df.isna().sum() > 0]

CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64

In [24]:
# Define categorical and continuous variables
dep_var = 'Sales'

cat_names = ['Store', 'DayOfWeek', 'StateHoliday', 'SchoolHoliday', 'StoreType', 
             'Assortment', 'Promo', 'Promo2', 'PromoInterval', 'Month', 'Day', 'Year', 'Week', 'Dayofweek']

cont_names = ['Customers', 'Open', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
              'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']

procs = [Categorify, FillMissing, Normalize]

## 5. Summary
- Date features extracted
- Missing values handled (with fastai 'FillMissing')
- Feature types split into categorical and continuous

## 6. Create TabularDataLoaders
We'll use fastai's `TabularDataLoaders` to:
- Apply preprocessing (categorify, fillmissing, normalize)
- Build training/validation splits
- Preview batches before model training

In [25]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Random train/valid split
splits = RandomSplitter(seed=42)(df)

# Create DataLoaders
dls = TabularDataLoaders.from_df(
    df, 
    procs=procs,
    cat_names=cat_names,
    cont_names=cont_names,
    y_names=dep_var,
    splits=splits,
    bs=64
)

dls.show_batch(max_n=10)

,Store,DayOfWeek,StateHoliday,SchoolHoliday,StoreType,Assortment,Promo,Promo2,PromoInterval,Month,Day,Year,Week,Dayofweek,CompetitionDistance_na,CompetitionOpenSinceMonth_na,CompetitionOpenSinceYear_na,Promo2SinceWeek_na,Promo2SinceYear_na,Customers,Open,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2SinceWeek,Promo2SinceYear,Sales
0,636,1,0,0,c,a,1,1,"Jan,Apr,Jul,Oct",10,21,2013,43,0,False,False,False,False,False,924.999989,1.0,720.000220,10.0,2004.0,13.0,2010.0,8432.0
1,1059,5,0,0,c,a,1,0,#na#,8,30,2013,35,4,False,False,False,True,True,1346.000018,1.0,3379.999930,4.0,2013.0,22.0,2012.0,10871.0
2,15,4,0,0,d,c,1,1,"Jan,Apr,Jul,Oct",5,30,2013,22,3,False,False,False,False,False,785.999996,1.0,4110.000012,3.0,2010.0,14.0,2011.0,8458.0
3,227,2,0,0,a,a,1,0,#na#,4,14,2015,16,1,False,True,True,True,True,631.000000,1.0,2369.999969,8.0,2010.0,22.0,2012.0,7155.0
4,1060,5,0,0,a,c,1,1,"Feb,May,Aug,Nov",2,21,2014,8,4,False,True,True,False,False,899.000001,1.0,3429.999978,8.0,2010.0,31.0,2013.0,7461.0
5,796,6,0,0,a,c,0,0,#na#,9,20,2014,38,5,False,False,False,True,True,448.999993,1.0,7180.000024,11.0,2012.0,22.0,2012.0,4268.0
6,210,3,0,0,d,a,0,1,"Feb,May,Aug,Nov",5,13,2015,20,2,False,False,False,False,False,341.999996,1.0,970.000001,11.0,1999.0,5.0,2013.0,3382.0
7,544,5,0,1,a,a,1,1,"Jan,Apr,Jul,Oct",7,31,2015,31,4,False,False,False,False,False,2050.000036,1.0,249.999952,12.0,2001.0,13.0,2010.0,17626.0
8,307,2,0,0,a,a,0,0,#na#,3,11,2014,11,1,False,False,False,True,True,292.000005,1.0,18660.000108,9.0,2002.0,22.0,2012.0,2157.0
9,505,4,0,0,a,a,1,1,"Feb,May,Aug,Nov",1,10,2013,2,3,False,True,True,False,False,654.000000,1.0,350.000048,8.0,2010.0,5.0,2013.0,5424.0


In [26]:
# Save a small sample (optional, for reference/debugging)
df.head(100).to_csv('../data/rossmann/rossmann_sample.csv', index=False)
print("Sample saved to data/rossmann/rossmann_sample.csv")

Sample saved to data/rossmann/rossmann_sample.csv


In [27]:
# Best practice: save full TabularPandas (optional upgrade)
to = TabularPandas(
    df,
    procs=procs,
    cat_names=cat_names,
    cont_names=cont_names,
    y_names=dep_var,
    splits=splits
)

import pickle

with open('../data/rossmann_tabular.pkl', 'wb') as f:
    pickle.dump(to, f)

## 7. Results

- Loaded and merged `train.csv` and `store.csv` on `Store`
- Parsed `Date` column and extracted date-based features (`Year`, `Month`, `Week`, etc.)
- Handled missing values using fastai's `FillMissing` processor
- Defined `cat_names` and `cont_names` for model inputs
- Created `TabularDataLoaders` with randomized training/validation split
- Previewed a clean batch with categorical encoding and normalization applied
- Saved a snapshot of the processed data as `rossmann_sample.csv`

## 8. Summary

- Preprocessing with fastai’s `TabularDataLoaders` allowed for seamless handling of categorical, continuous, and missing values
- `add_datepart()` extracted useful temporal features from `Date` for tabular learning
- Fastai’s `FillMissing` created companion `_na` columns, preserving interpretability
- Defined structured lists of categorical and continuous variables for use in model training
- Dataset is now fully processed and ready for deep learning — next step: build and train the model